Assignment 1 k-NN DTW
=

Task 1-1 implement a 1-NN DTW classifier
--

In [1]:
# DTW background
import numpy as np

def dtw(s, t, window=4):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)]) # warping cannot be less than the difference in lengths. 
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix[-1,-1]

In [2]:
#DTW classifier function

def DTWClassifier(x_train,y_train,x_test,w):
    y=np.array([])
    distance=[]
    
    for n in range(len(x_test)):#测试集的每一行
        for m in range(len(x_train)):#与训练集的每一行计算距离
            distance.append(dtw(x_test[n],x_train[m],w))
        num=distance.index(min(distance)) #get the row which has shortest distance to the test row
        y=np.append(y, y_train[num]) # use this y as the predict result
        distance.clear()
        
    return y

Task 1-2 Test the performance of DTW classifier
--

In [3]:
import pandas as pd

#read the file
file_path = "UMD_TEST.txt"
file = pd.read_csv(file_path,sep='\s+',header=None)
#将列名转化为str，方便后续操作指定
file.columns = file.columns.astype(str)
file.head()

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,1.0,0.017644,0.030949,0.050555,0.044484,0.053277,0.041576,0.030947,0.027086,0.013764,...,0.024575,0.033780,0.026589,0.013932,0.024928,0.022589,0.038248,0.049838,0.053419,0.040420
1,1.0,0.041296,0.003551,0.027470,0.013158,0.009571,0.008074,0.043743,0.040592,0.012190,...,0.060539,0.046991,0.023586,0.001562,-0.002196,0.036730,0.039027,0.007754,0.004697,0.031440
2,1.0,-0.000720,0.013283,0.029450,0.045201,0.006317,0.018805,0.028901,0.013832,0.015240,...,0.016442,0.039508,0.015171,0.034708,0.010835,0.002942,0.006924,0.029502,0.040786,0.023144
3,1.0,0.005201,0.013363,0.025733,0.026653,0.038946,0.012494,0.028303,0.032011,0.009467,...,0.006383,0.037448,0.044335,0.011143,-0.003624,0.001467,0.020991,0.027675,0.001621,0.015858
4,1.0,0.022926,0.027036,0.011668,0.019500,0.036049,-0.001297,0.019717,0.039583,0.020628,...,0.026997,0.036653,0.018117,0.018314,0.012536,0.040599,0.016590,0.032730,0.002498,0.011260


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#split test-set & train-set
file_train, file_test= train_test_split(file, test_size=0.2,random_state=42)
y_train=file_train.pop('0').values
x_train=file_train.values
y_test =file_test.pop('0').values
x_test =file_test.values


In [6]:
#predict y_test
import time
start_time=time.time()

y_pred_dtw=DTWClassifier(x_train,y_train,x_test,4)

end_time=time.time()
cost_time_1=end_time-start_time
print('1-NN DTW needs',cost_time_1,'seconds to predict the result')

#compute the accuracy score of DTW classifier
acc_dtw = accuracy_score(y_test, y_pred_dtw)

print('1-NN DTW has',acc_dtw,'accuracy')

1-NN DTW needs 56.86910700798035 seconds to predict the result
1-NN DTW has 0.9655172413793104 accuracy


Task 1-3 Compare the 1-NN DTW performance with 1-NN Euclidean
--

In [9]:
from sklearn.metrics import DistanceMetric
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import distance


# use scikit-learn Euclidean distance to replace 
def EucClassifier(x_train,y_train,x_test):
    y=np.array([])
    dist=[]
    #dist = DistanceMetric.get_metric('euclidean')
        
    for n in range(len(x_test)):# 测试集的每一行
        for m in range(len(x_train)):# 与训练集的每一行计算距离
            dist.append(float(euclidean_distances(np.expand_dims(x_test[n],0),np.expand_dims(x_train[m],0)).flatten()))
        num=dist.index(min(dist)) #get the row from train-set which has shortest distance to the test row
        y=np.append(y, y_train[num]) # use this y as the predict result
        dist.clear()
        
    return y

start_time=time.time()
y_pred_euc=EucClassifier(x_train,y_train,x_test)
end_time=time.time()
cost_time_2=end_time-start_time
print('1-NN Euclidean needs',cost_time_2,'seconds to predict the result')
acc_euc = accuracy_score(y_test, y_pred_euc)

print('1-NN Euclidean has',acc_euc,'accuracy')

1-NN Euclidean needs 0.4299759864807129 seconds to predict the result
1-NN Euclidean has 0.896551724137931 accuracy


The accuracy for 1-NN Euclidean is 0.897, which costs 0.43 seconds.
--
The accuracy for 1-NN DTW is 0.965, which costs 56 seconds.
--

Task 1-4 Find the best value for the window parameter
--

In [10]:
from sklearn.metrics import  accuracy_score

for w in range(1,10):
    y_pred=DTWClassifier(x_train,y_train,x_test,w)
    acc = accuracy_score(y_test, y_pred)
    print(w, acc)

1 0.8620689655172413
2 0.8620689655172413
3 0.8620689655172413
4 0.9655172413793104
5 0.9655172413793104
6 0.9655172413793104
7 0.9655172413793104
8 0.9655172413793104
9 0.9655172413793104


When windows=4, we get the highest accurancy.

Before this, I try the windows values from 1 to 150(cost really long time), the accurancy increased from 1 to 4, but stopped after 5.

Then I use windows=4 in all DTW function in this assignment.

Task 2-1 Use DTW code as the metric parameter in scikit-learn k-NN  classifier
--

In [11]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=1, metric=dtw)
clf.fit(x_train,y_train)


KNeighborsClassifier(metric=<function dtw at 0x7f8344d519d0>, n_neighbors=1)

Task 2-2 Test the performance : KNN with DTW vs. DTWClassifier
--

In [12]:

start_time=time.time()
y_pred_kd = clf.predict(x_test)
end_time=time.time()
cost_time=end_time-start_time
print('KNN DTW needs',cost_time,'seconds to predict the result')

acc_kd = accuracy_score(y_test, y_pred_kd)
print('the accuracy for KNN with DTW is',acc_kd)
print('the accuracy for DTWClassifier is',acc_dtw)

KNN DTW needs 57.986408948898315 seconds to predict the result
the accuracy for KNN with DTW is 0.9655172413793104
the accuracy for DTWClassifier is 0.9655172413793104


the accuracy for 1-NN DTW is equal to KNN with DTW.

Task 2-3 Compare with k-NN Euclidean
--

In [15]:
for n in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=n)
    start_time=time.time()
    knn.fit(x_train,y_train)
    y_pred_knn = knn.predict(x_test)
    end_time=time.time()
    cost_time=end_time-start_time
    acc_knn = accuracy_score(y_test, y_pred_knn)
    print('when n_neighbors=',n,'the accuracy is',acc_knn,'k-NN Euclidean costs',cost_time,'seconds')

when n_neighbors= 1 the accuracy is 0.896551724137931 k-NN Euclidean costs 0.0045261383056640625 seconds
when n_neighbors= 2 the accuracy is 0.7586206896551724 k-NN Euclidean costs 0.0063130855560302734 seconds
when n_neighbors= 3 the accuracy is 0.9310344827586207 k-NN Euclidean costs 0.005640983581542969 seconds
when n_neighbors= 4 the accuracy is 0.896551724137931 k-NN Euclidean costs 0.003136873245239258 seconds
when n_neighbors= 5 the accuracy is 0.9310344827586207 k-NN Euclidean costs 0.0034339427947998047 seconds
when n_neighbors= 6 the accuracy is 0.8620689655172413 k-NN Euclidean costs 0.004797935485839844 seconds
when n_neighbors= 7 the accuracy is 0.8275862068965517 k-NN Euclidean costs 0.003407001495361328 seconds
when n_neighbors= 8 the accuracy is 0.8620689655172413 k-NN Euclidean costs 0.0037081241607666016 seconds
when n_neighbors= 9 the accuracy is 0.8620689655172413 k-NN Euclidean costs 0.003336191177368164 seconds
when n_neighbors= 10 the accuracy is 0.86206896551724

The accuracy for 1-NN DTW classifier is 0.966;

The accuracy for 1-NN Euclidean classifier is 0.897;

The accuracy for k-NN classifier with DTW distance is 0.966(n_neighbors=1);

The accuracy for k-NN classifier with Euclidean distance is 0.93(n_neighbors=3).

When predict this time series, DTW distance has better accuracy performance than Euclidean distance.
--

But when computing DTW distance, it needs much more time than Euclidean distance.
--